**Goal:** Create a scoring system for each batch effect correction evaluation metric: 

- Batch QC
- PCA
- DSC
- LFC
- DGE across dataset


In [73]:
# Import libraries
import pandas as pd
import math
import numpy as np
from scipy.stats import kurtosis, skew
import seaborn as sns

In [74]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [75]:
corrDir = 'path/to/Batch_corrected_data/'

In [76]:
# Read in a metadata table with all variables
meta = pd.read_csv('path/to/all_metadata_Proj2.csv')
meta.head()

,Sample,age,animalreturn,dataset,condition,duration,gender,libPrep,mission,preservation,seqFacility,seqParameters,strain
0,GLDS_137_Mmus_BAL.TAL_LVR_FLT_Rep1_F1,age_12w,ISST,GLDS_137,FLT,dur_42d,female,ribodepleted,RR3,carcass,UCDavis,PE_150bp_100Mreads,BALBcT
1,GLDS_137_Mmus_BAL.TAL_LVR_FLT_Rep2_F2,age_12w,ISST,GLDS_137,FLT,dur_42d,female,ribodepleted,RR3,carcass,UCDavis,PE_150bp_100Mreads,BALBcT
2,GLDS_137_Mmus_BAL.TAL_LVR_FLT_Rep3_F3,age_12w,ISST,GLDS_137,FLT,dur_42d,female,ribodepleted,RR3,carcass,UCDavis,PE_150bp_100Mreads,BALBcT
3,GLDS_137_Mmus_BAL.TAL_LVR_FLT_Rep4_F4,age_12w,ISST,GLDS_137,FLT,dur_42d,female,ribodepleted,RR3,carcass,UCDavis,PE_150bp_100Mreads,BALBcT
4,GLDS_137_Mmus_BAL.TAL_LVR_FLT_Rep5_F5,age_12w,ISST,GLDS_137,FLT,dur_42d,female,ribodepleted,RR3,carcass,UCDavis,PE_150bp_100Mreads,BALBcT


### BatchQC

BatchQC is exclusively visualizations and creates no quantifiable output. Of all the visualizations, the skew and kurtosis are unique among all the other evaluations we're performing. 

Therefore, need to re-generate the skew and kurtosis values in Python for all corrected datasets. 

Skew and Kurtosis are two ways of testing the **normality** of a distribution (in this case, we are testing the normality of the gene expression distribution for each sample). 

- Skewness: a measure of how far the shape of the distribution deviates from symmetry around its center. If the skewness = 0, the distribution is exactly symmetrical.
- Kurtosis: a measure of how much weight is at the tails of the distribution relative to the weight around the center of the distribution. That is, how much does the central peak rise above the tails. The kurtosis of a normal distribution = 0. 

>Citations: https://academic.oup.com/biostatistics/article/16/4/627/254836?login=true (cited in the BatchQC paper)
https://community.gooddata.com/metrics-and-maql-kb-articles-43/normality-testing-skewness-and-kurtosis-241

From citation above (modifications mine): "From [skew and kurtosis] summaries, one can assess how the data deviates from hypothesized distributional assumptions and also assess how certain genes [samples] deviate from the typical gene [sample] in a given dataset. These deviant genes [samples] may be classified as volatile (or outliers) or interesting genes. Perhaps they may be genes [samples] affected by batch effects ..."

Therefore: we want the winning post-correction dataset to have skew and kurtosis values which are _most similar_ BETWEEN batch types compared to the pre-correction dataset.

So: For each batch-method combo: For each sample, calculate abs value of ideal (0) minus precorrected, and abs value of ideal (0) minus corrected, and then calculate [NOT abs value] of the precorrected vector minus the corrected vector (1 value for each sample). For each technical variable: Within each batch type, take the mean of all diffs, then take the mean across all batch types (so get 5 values for skew and 5 for kurtosis).  

10 values per batch-method combo ([list of 5 kurtosis], [list of 5 skew]). Want to _maximize_ all (because it's the difference between pre- and post-correction). 

In [77]:
def skew_and_kurtosis(uncorrExp, corrExp, metaTable):
    
    # Calculate uncorrected kurtosis and skew
    uncorr = pd.DataFrame()
    uncorr['kurtosis'] = kurtosis(uncorrExp)
    uncorr['skew'] = skew(uncorrExp)
    #uncorr[batch] = metaTable[batch]
    uncorr.index = metaTable['Sample']
    
    # Calculate corrected kurtosis and skew
    corr = pd.DataFrame()
    corr['kurtosis'] = kurtosis(corrExp)
    corr['skew'] = skew(corrExp)
    #corr[batch] = metaTable[batch]
    corr.index = metaTable['Sample']
    
    # Add columns with absolute value of difference from 0 (which is just abs() of the value itself)
    uncorr['zero_diff_kurtosis'] = abs(uncorr['kurtosis'])
    uncorr['zero_diff_skew'] = abs(uncorr['skew'])
    corr['zero_diff_kurtosis'] = abs(corr['kurtosis'])
    corr['zero_diff_skew'] = abs(corr['skew'])
    
    # Create df with diffs between precorr and corr (not abs value; want to maximize)
    diff_df = pd.DataFrame()
    diff_df['kurtosis_diffs'] = uncorr['zero_diff_kurtosis'] - corr['zero_diff_kurtosis']
    diff_df['skew_diffs'] = uncorr['zero_diff_skew'] - corr['zero_diff_skew']
    diff_df.index = metaTable['Sample']
    
    
    
    final_kurt = [] # lists to hold the mean kurt and skew diff values for each tech variable below (these lists should each have 5 final values)
    final_skew = []
    for techVar in ['libPrep', 'mission', 'preservation', 'seqFacility', 'seqParameters']:

        k = [] # lists to hold mean kurt and skew diff values for each batch type within this batch
        s = []
        
        grouped = pd.DataFrame(metaTable.groupby(techVar).agg(lambda x: x.tolist())['Sample']) # group all samples into batch types for the batch
        
        for batchType in grouped.index: # iterate through the batch types (e.g. different missions)
            k.append(np.mean(diff_df.loc[list(grouped.loc[batchType])[0]]['kurtosis_diffs']))
            s.append(np.mean(diff_df.loc[list(grouped.loc[batchType])[0]]['skew_diffs']))
            
        final_kurt.append(np.mean(k))
        final_skew.append(np.mean(s))

    
    zipped = list(zip(final_kurt, final_skew))
    
    return ([list(x) for x in zipped]) # return a two-item list for each of the 5 variables: [k, s]
    

In [79]:
## Read in uncorrected gene expression
uncorr_exp = pd.read_csv('path/to/Proj2_Normalized_Counts.csv', index_col=0)
uncorr_exp = uncorr_exp.reindex(meta['Sample'], axis=1) # reorder columns according to meta

In [80]:
### Calculate the BatchQC scores 

batchqc_vector = [] # a tuple of 2 lists ([5 values for kurtosis], [5 values for skew]) for each batch-method pair

for batch in ['libPrep_as_batch/', 'mission_as_batch/']:
    for corrMethod in ['ComBat_seq/','ComBat_standard/','MBatch_AN/','MBatch_EB/','MBatch_MP/']:
        corr_exp = pd.read_csv(corrDir+batch+corrMethod+'Corrected_Counts/Corrected_Counts.csv', index_col=0)
        corr_exp = corr_exp.reindex(meta['Sample'], axis=1) # reorder columns according to meta
        
        
        batchqc_vector.append(skew_and_kurtosis(uncorr_exp, corr_exp, meta)) 

In [81]:
len(batchqc_vector)

10

In [82]:
len(batchqc_vector[0])

5

In [83]:
len(batchqc_vector[0][0])

2

In [84]:
batchqc_vector

[[[2128.978565204061, 10.478337653491538],
  [2732.3432053918636, 14.527088919442434],
  [3682.8308767718236, 17.98644734417225],
  [3568.329865687373, 17.564133814820842],
  [2417.7364844304834, 12.469828526140732]],
 [[2778.5973191368726, 9.714860985983748],
  [1938.8691897082124, 7.058153012205305],
  [1749.143305031755, 6.676280100999961],
  [1523.9127654850442, 6.0175804981410925],
  [2150.0809458467425, 7.85536492312464]],
 [[1232.8078794275993, 5.633040009423116],
  [1947.0989933418489, 10.031021425378308],
  [1768.0346649206983, 9.051641527104074],
  [1814.5714341340606, 9.301005656529917],
  [1440.259048214557, 7.209727993198283]],
 [[2776.0425094020993, 9.71337176854651],
  [1933.8776117407267, 7.040718020923251],
  [1744.4726888691082, 6.657928503690852],
  [1518.7476489863052, 5.995330092054004],
  [2146.6250366258123, 7.844939621694155]],
 [[-2491.2859915329473, -9.032886088327409],
  [-3136.5220212869885, -10.511462965845753],
  [-4215.942578534743, -15.456386883430072],


### PCA
<!-- OLD: For each batch (e.g. polyA and ribodeplete), calculate the distance between all samples before and after correction. Take the difference between pre/post correct. Take the mean 


Based on the batch type, what is the difference in distance between all samples in each batch before and after correction. After correction this should be maximized. Take the average across the plot so there is just 1 value.
 -->


<!-- OLD #2: Want to minimize the distances between the samples of different batches, post correction. So, for whichever batch is being corrected, calculate the distance between all pairwise samples from different batches. Do this for pre and post correction datasets. Take the absolute value of the differences between pre and post, and take the mean overall. 

1 value for each batch-method combo. Want to _maximize_ this value (because we are comparing with pre-correction).
 -->
 

All of this takes place with corrected data, because we don't need to know what the uncorrected data looked like in order to max/min optimize the within-batch and within-condition distances. In fact comparing to uncorrected might make a bad method look good if there's a sufficiently large difference from uncorrected.
 
For each batch-method combo: For each sample, calculate 1) the average distance it is away from all other samples NOT from the same technical batch (want to minimize this, but take negative so maximize); and 2) the average distance it is away from all other samples NOT from the same FLT/GC condition (want to maximize this). Then take the MEDIAN across all samples for both measurements. Do this within each technical variable. 



So, for each of the 15 batch-method combos, get a list of 5 tuples (one for each tech variable). The tuples will be: x=the negative median across all samples of the average distance to all other samples outside of the same batch (maximize), and y=the median across all samples of the average distance to all other samples outside of the same condition (maximize). 
 

 
 

In [85]:
def getDistances(corrPC, metaTable):#(uncorrPC, corrPC, metaTable, batch):
    '''
    --- This description needs to be updated ---
    PC1 and PC2:
    Function to calculate the average distance between all samples not from the same batch.
    Take the difference of corrected - uncorrected.
    Take the average across the entire dataset.
    
    Inputs: 
    - corrPC = PC matrix of corrected data (this will be unique for each batch-method combination)
    - metaTable = metadata table relating samples and batches, subset to just the relevant batch column (this will be unique to each batch)
    '''
    
    def dist(x1, y1, x2, y2):
        '''Distance between 2 points in an xy plane.'''
        
        d = math.sqrt( (x2-x1)**2 + (y2-y1)**2 )
        return d
        
    
    # list to hold 5 lists with 2 values (negative median distance to other batches, median distance to other conditions)
    finalDists = []
    
    # create a dictionary of sample:condition
    condDict = meta.set_index('Sample')['condition'].to_dict() 
        
    ### Iterate through each of 5 tech variables ###
    for techVar in ['libPrep', 'mission', 'preservation', 'seqFacility', 'seqParameters']:
        # create a dictionary of sample:batch
        batchDict = meta.set_index('Sample')[techVar].to_dict()
    
    
        meanBatchDists = [] # list to hold the mean distance to non-same-batch samples for all samples (112)
        meanCondDists = [] # list to hold the mean distance to non-same-cond samples for all samples (112)
        
        ### Iterate through each of 112 samples ###
        for s1 in batchDict: 
            otherBatches = [x for x in batchDict if batchDict[x] != batchDict[s1]]
            otherCond = [x for x in condDict if condDict[x] != condDict[s1]]
            
            # calculate distance between s1 and all samples in otherBatches and otherCond
            batDists = [] # lists to hold all pairwise distances, which will then be averaged
            condDists = []
            
            for s2 in otherBatches:
                batDists.append(dist(corrPC['PC1'].loc[s1],
                    corrPC['PC2'].loc[s1],
                    corrPC['PC1'].loc[s2],
                    corrPC['PC2'].loc[s2]))
            for s2 in otherCond:
                condDists.append(dist(corrPC['PC1'].loc[s1],
                    corrPC['PC2'].loc[s1],
                    corrPC['PC1'].loc[s2],
                    corrPC['PC2'].loc[s2]))
            
            meanBatchDists.append(np.mean(batDists)) # take the average of all neighbors for that sample, append to mean lists
            meanCondDists.append(np.mean(condDists))
            
        
        finalDists.append([-np.median(meanBatchDists), np.median(meanCondDists)])
    
    print(len(finalDists))

    return finalDists
    

In [87]:
### Calculate the PCA scores 

pca_vector = [] # list to hold vectors for each of 15 batch-method combos

for batch in ['libPrep_as_batch/', 'mission_as_batch/']:
    for corrMethod in ['ComBat_seq/','ComBat_standard/','MBatch_AN/','MBatch_EB/','MBatch_MP/']:
        corr_pca = pd.read_csv(corrDir+batch+corrMethod+'PCA/PCA_table.csv', index_col=0)
        pca_vector.append(getDistances(corr_pca, meta))


5
5
5
5
5
5
5
5
5
5


In [88]:
len(pca_vector)

10

In [89]:
len(pca_vector[0])

5

In [90]:
len(pca_vector[0][0])

2

In [91]:
pca_vector

[[[-72.13313817411947, 77.03286680161123],
  [-83.691916653025, 77.03286680161123],
  [-82.28294878435311, 77.03286680161123],
  [-80.79162907607339, 77.03286680161123],
  [-80.54561433839177, 77.03286680161123]],
 [[-72.03972280269909, 76.65703859598875],
  [-84.19901805374496, 76.65703859598875],
  [-82.02617604859319, 76.65703859598875],
  [-78.96273836043508, 76.65703859598875],
  [-77.72211456510911, 76.65703859598875]],
 [[-95.92866702291852, 109.53935538474386],
  [-121.89266397890032, 109.53935538474386],
  [-128.81284403747918, 109.53935538474386],
  [-135.76305655412585, 109.53935538474386],
  [-131.87441383710666, 109.53935538474386]],
 [[-71.05433292096208, 75.4159071520525],
  [-83.45117575283575, 75.4159071520525],
  [-80.93438395270506, 75.4159071520525],
  [-78.29677237396395, 75.4159071520525],
  [-76.96462439591835, 75.4159071520525]],
 [[-217.65887507126098, 243.30980589985984],
  [-257.63942976666044, 243.30980589985984],
  [-289.6228226693177, 243.30980589985984],


### DSC
Difference in DSC for "condition" (FLT/GC) and each of the 5 technical variables between corrected-uncorrected.

Maximize post correction "condition" DSC difference and "minimize" all technical variable differences (because we want the technical DSCs to be smaller after correction so the difference will be negative) - (except take negative so maximize).

For each batch-method combo, get a list of 6 values: [condition, 5 tech variables]

In [92]:
# Uncorrected DSC values
uncorr_dsc = pd.read_csv('/path/to/DSC_table.csv', index_col=0)
uncorr_dsc

,age,animalreturn,dataset,condition,duration,gender,libPrep,mission,preservation,seqFacility,seqParameters,strain
Overall DSC,0.170228,0.188016,0.19608,0.105764,0.29203,0.049936,0.082513,0.183795,0.177257,0.134238,0.164404,0.167407


In [93]:
### Calculate the DSC scores 

dsc_vector = [] # a tuple for each batch-method pair: the first is the corr-uncorr condition, and the second is the corr-uncorr batch DSC

# Iterate through all 15 batch-method combos
for batch in ['libPrep_as_batch/', 'mission_as_batch/']:
    for corrMethod in ['ComBat_seq/','ComBat_standard/','MBatch_AN/','MBatch_EB/','MBatch_MP/']:
        
        corr_dsc = pd.read_csv(corrDir+batch+corrMethod+'DSC/DSC_table.csv', index_col=0)
        
        diffsList = [] # list to hold the 5 DSC diff values
        
        # Iterate through all 5 technical variables and append each DSC diff value
        for techVar in ['libPrep', 'mission', 'preservation', 'seqFacility', 'seqParameters']:
        
            diffsList.append(-(corr_dsc[techVar].loc['Overall DSC']-uncorr_dsc[techVar].loc['Overall DSC']))

        dsc_vector.append(diffsList)

In [94]:
len(dsc_vector)

10

In [95]:
len(dsc_vector[0])

5

In [96]:
dsc_vector

[[0.0291866500097831,
  0.03396085434889798,
  0.034968871452175004,
  0.010991684466330004,
  0.038094128356158996],
 [0.002593068280833291,
  0.019235656788409994,
  0.022004936769948996,
  0.021722693010243002,
  0.029604937804107],
 [0.0365692443611619,
  -0.097572430740337,
  0.1194520079658304,
  -0.113292573991575,
  -0.132279890187004],
 [0.0705058513837436,
  -0.219261778489742,
  0.02981857319884501,
  -0.031965328663159986,
  -0.012993578075822992],
 [0.007894540706487096,
  -0.226945515342322,
  0.037515560182595,
  -0.05074821673323199,
  -0.021115825099617985],
 [-0.06132827018437571,
  -0.10016512503951303,
  -0.037079878705196995,
  0.006460753280899012,
  -0.08152276491423399],
 [0.0383720473302919,
  0.06183281432372699,
  0.029374580980664006,
  0.0531337604978174,
  0.07736011576511491],
 [-0.2045898953888467,
  0.11711067886384278,
  0.0962190913629878,
  0.07532576524183471,
  -0.13710695391188402],
 [-0.16949847402453372,
  0.1382328450505605,
  0.065666504629351

### LFC

For each batch-method combo: For each of the 5 tech variables: What is the average improvement in LFC for pairwise datasets which come from DIFFERENT batches in this tech variable? 


Scale the uncorr and corr matrices to 0-1 to get positive numbers (maximize result). 


Subtract corr - uncorr. 


Then, for each tech variable, set any pairwise nodes that come from the same batch to 0 (just to get rid of them). (Do this by creating a matrix of 0s and 1s for each tech variable and multiple the diff'd matrix by it: e.g. 2 datasets from the same mission = 0, 2 datasets from different missions = 1). Take the mean of the rest of the matrix to get the average improvement in LFC across datasets from different batches. 


Result: A list of 15 lists. Each list has 5 values: the mean improvement for each tech variable. Maximize all.






In [97]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object (default feature reange = (0,1))
scaler = MinMaxScaler()

In [98]:
# Read in the batches matrices (manually populated in Excel)

In [99]:
lp_w = pd.read_csv('/path/to/lfc_pairwise_matches_libprep.csv', index_col=0)

In [100]:
sf_w = pd.read_csv('/path/to/lfc_pairwise_matches_seqFacility.csv', index_col=0)

In [101]:
m_w = pd.read_csv('/path/to/lfc_pairwise_matches_mission.csv', index_col=0)

In [102]:
p_w = pd.read_csv('/path/to/lfc_pairwise_matches_preservation.csv', index_col=0)

In [103]:
sp_w = pd.read_csv('/path/to/lfc_pairwise_matches_seqParameters.csv', index_col=0)

In [104]:
m_w

,GLDS-47,GLDS-48_I,GLDS-48_C,GLDS-137,GLDS-168,GLDS-173,GLDS-242,GLDS-245_LAR,GLDS-245_ISST
GLDS-47,0,1,1,1,1,1,1,1,1
GLDS-48_I,1,0,0,0,1,1,1,1,1
GLDS-48_C,1,0,0,0,1,1,1,1,1
GLDS-137,1,0,0,0,0,1,1,1,1
GLDS-168,1,0,0,0,0,1,1,1,1
GLDS-173,1,1,1,1,1,0,1,1,1
GLDS-242,1,1,1,1,1,1,0,1,1
GLDS-245_LAR,1,1,1,1,1,1,1,0,0
GLDS-245_ISST,1,1,1,1,1,1,1,0,0


In [105]:
# Uncorrected LFC values
uncorr_lfc = pd.read_csv('/path/to/cor_lfc.csv', index_col=0)
uncorr_lfc

,GLDS-47,GLDS-48_I,GLDS-48_C,GLDS-137,GLDS-168,GLDS-173,GLDS-242,GLDS-245_LAR,GLDS-245_ISST
GLDS-47,1.00,-0.07,0.01,-0.03,-0.04,0.05,0.16,-0.01,0.01
GLDS-48_I,-0.07,1.00,0.04,-0.01,0.19,0.03,-0.02,0.08,-0.08
GLDS-48_C,0.01,0.04,1.00,-0.01,0.27,-0.07,-0.11,-0.04,0.17
GLDS-137,-0.03,-0.01,-0.01,1.00,0.09,0.03,-0.06,0.00,0.01
GLDS-168,-0.04,0.19,0.27,0.09,1.00,-0.08,-0.19,0.20,0.26
GLDS-173,0.05,0.03,-0.07,0.03,-0.08,1.00,0.09,0.08,-0.16
GLDS-242,0.16,-0.02,-0.11,-0.06,-0.19,0.09,1.00,0.11,-0.01
GLDS-245_LAR,-0.01,0.08,-0.04,0.00,0.20,0.08,0.11,1.00,0.16
GLDS-245_ISST,0.01,-0.08,0.17,0.01,0.26,-0.16,-0.01,0.16,1.00


In [106]:
# Scale uncorr LFC between 0-1 
scaler.fit(uncorr_lfc)
uncorr_lfc_sc = pd.DataFrame(scaler.transform(uncorr_lfc))
uncorr_lfc_sc

,0,1,2,3,4,5,6,7,8
0,1.000000,0.009259,0.108108,0.028302,0.126050,0.181034,0.294118,0.028846,0.146552
1,0.000000,1.000000,0.135135,0.047170,0.319328,0.163793,0.142857,0.115385,0.068966
2,0.074766,0.111111,1.000000,0.047170,0.386555,0.077586,0.067227,0.000000,0.284483
3,0.037383,0.064815,0.090090,1.000000,0.235294,0.163793,0.109244,0.038462,0.146552
4,0.028037,0.250000,0.342342,0.141509,1.000000,0.068966,0.000000,0.230769,0.362069
5,0.112150,0.101852,0.036036,0.084906,0.092437,1.000000,0.235294,0.115385,0.000000
6,0.214953,0.055556,0.000000,0.000000,0.000000,0.215517,1.000000,0.144231,0.129310
7,0.056075,0.148148,0.063063,0.056604,0.327731,0.206897,0.252101,1.000000,0.275862
8,0.074766,0.000000,0.252252,0.066038,0.378151,0.000000,0.151261,0.192308,1.000000


In [107]:
### Calculate the LFC scores 

lfc_vector = [] # list of 15 lists (one for each batch-method combo); each list with 5 values (one for each tech var)

for batch in ['libPrep_as_batch/', 'mission_as_batch/']:
    for corrMethod in ['ComBat_seq/','ComBat_standard/','MBatch_AN/','MBatch_EB/','MBatch_MP/']:
        
        mean_lfcs = [] # list to hold the 5 mean LFC improvements for each tech variable
        
        corr_lfc = pd.read_csv(corrDir+batch+corrMethod+'LFC/cor_lfc.csv', index_col=0) 
        scaler.fit(corr_lfc)
        corr_lfc_sc = pd.DataFrame(scaler.transform(corr_lfc)) # scale the corrected LFC matrix
        
        diff_lfc = corr_lfc_sc - uncorr_lfc_sc # subtract the uncorr matrix from the corrected
        
        # Iterate through all 5 technical variables
        for techVar in ['libPrep', 'mission', 'preservation', 'seqFacility', 'seqParameters']:
            
            if techVar == 'libPrep':
                batches = lp_w
            if techVar == 'mission':
                batches = m_w
            if techVar == 'preservation':
                batches = p_w
            if techVar == 'seqFacility':
                batches = sf_w
            if techVar == 'seqParameters':
                batches = sp_w
            
            mat = corr_lfc * batches # multiply the diff matrix by the batches matrix, to get only pairwise comparisons from different batches
        
            mean_lfcs.append(np.mean(np.mean(mat))) # calculate the mean (first gets row means, then full mean)
            
        lfc_vector.append(mean_lfcs)

In [108]:
len(lfc_vector)

10

In [109]:
len(lfc_vector[0])

5

In [110]:
lfc_vector

[[0.007160493827160493,
  0.0006172839506172847,
  -0.012839506172839502,
  0.011111111111111112,
  0.008148148148148147],
 [0.0069135802469135815,
  0.01037037037037037,
  0.0019753086419753083,
  0.013580246913580249,
  0.014567901234567903],
 [0.007160493827160493,
  0.008641975308641976,
  0.002222222222222221,
  0.01037037037037037,
  0.013086419753086422],
 [0.007160493827160495,
  0.011111111111111113,
  0.0032098765432098763,
  0.013333333333333332,
  0.014814814814814817],
 [0.0064197530864197536,
  -0.0013580246913580227,
  -0.01234567901234568,
  0.005925925925925928,
  0.008641975308641978],
 [0.008395061728395062,
  -0.0012345679012345677,
  -0.0076543209876543195,
  0.011358024691358026,
  0.012098765432098764],
 [0.011604938271604939,
  0.0029629629629629646,
  -0.006666666666666667,
  0.01604938271604938,
  0.01802469135802469],
 [0.003456790123456791,
  -0.006296296296296296,
  -0.024444444444444446,
  0.015555555555555557,
  0.01382716049382716],
 [0.01012345679012346

### DGE across-dataset

Same as LFC, except this criterion already incorporates the uncorrected data so no need to subtract it.

In [114]:
### Calculate the DGE across scores 

corrDir = '/[path/to/Project-2_standard_corrections/'

dge_across_vector = [] # list of 15 lists (one for each batch-method combo); each list with 5 values (one for each tech var)

for batch in ['libPrep_as_batch/', 'mission_as_batch/']:
    for corrMethod in ['ComBat_seq','ComBat_standard','MBatch_AN','MBatch_EB','MBatch_MP']:
        
        mean_dges = [] # list to hold the 5 mean LFC improvements for each tech variable
        
        dge_across = pd.read_csv(corrDir+batch+'DEG_Comparisons/'+corrMethod+'_percent_overlap_table_updatedFactors.csv', index_col=0) 
              
        # Iterate through all 5 technical variables
        for techVar in ['libPrep', 'mission', 'preservation', 'seqFacility', 'seqParameters']:
            
            if techVar == 'libPrep':
                batches = lp_w
            if techVar == 'mission':
                batches = m_w
            if techVar == 'preservation':
                batches = p_w
            if techVar == 'seqFacility':
                batches = sf_w
            if techVar == 'seqParameters':
                batches = sp_w
            
            mat = np.array(dge_across) * np.array(batches) # multiply the diff matrix by the batches matrix, to get only pairwise comparisons from different batches - convert to numpy arrays because the column and row names are different
        
            mean_dges.append(np.mean(np.mean(mat))) # calculate the mean (first gets row means, then full mean)
            
        dge_across_vector.append(mean_dges)

In [115]:
len(dge_across_vector)

10

In [116]:
len(dge_across_vector[0])

5

In [117]:
dge_across_vector

[[6.067754262198707,
  9.35270919067215,
  6.403341171859689,
  5.506319811875366,
  8.251028806584362],
 [4.837595532039978,
  9.078619330676945,
  6.3338967274152465,
  6.379349293443946,
  7.458861345178219],
 [7.649911816578484,
  10.08969397086681,
  5.308274544385655,
  5.654802730420014,
  9.145396498791559],
 [6.895208700764254,
  12.173830208809633,
  8.667083088379385,
  7.982790624251529,
  9.7824672850393],
 [9.373162845385066,
  16.61536242297559,
  11.65895061728395,
  10.36058582097677,
  14.974783351840966],
 [1.4711934156378592,
  2.389090295468896,
  2.5269449343523425,
  1.8220219043264303,
  1.9763428919807513],
 [4.211493239271016,
  4.81508698586888,
  3.0015432098765435,
  3.4089288435996257,
  4.534002329784222],
 [1.6909171075837743,
  2.0158213033727437,
  0.6817068391142466,
  1.6813366864807195,
  1.829043917521284],
 [4.16005291005291,
  4.3925414244344285,
  2.648809523809524,
  3.4089288435996257,
  4.482562000566115],
 [17.263374485596707,
  31.917036819

### Create table

10 rows: one for each batch-method pair



In [118]:
vectors = pd.DataFrame(columns=['BatchQC','PCA','DSC','LFC','DGEacross'],
                      index=['ComBatseq_libprep',
                                'ComBat_libprep',
                                'AN_libprep',
                                'EB_libprep',
                                'MP_libprep',
                                'ComBatseq_mission',
                                'ComBat_mission',
                                'AN_mission',
                                'EB_mission',
                                'MP_mission'])

vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,NaN,NaN,NaN,NaN,NaN
ComBat_libprep,NaN,NaN,NaN,NaN,NaN
AN_libprep,NaN,NaN,NaN,NaN,NaN
EB_libprep,NaN,NaN,NaN,NaN,NaN
MP_libprep,NaN,NaN,NaN,NaN,NaN
ComBatseq_mission,NaN,NaN,NaN,NaN,NaN
ComBat_mission,NaN,NaN,NaN,NaN,NaN
AN_mission,NaN,NaN,NaN,NaN,NaN
EB_mission,NaN,NaN,NaN,NaN,NaN
MP_mission,NaN,NaN,NaN,NaN,NaN


Populate with the quantification vectors --- 

In [119]:
# BatchQC
i=0
for x in batchqc_vector:
    
    vectors['BatchQC'].loc[vectors.index[i]] = x
    
    i+=1

In [120]:
vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,"[[2128.978565204061, 10.478337653491538], [273...",NaN,NaN,NaN,NaN
ComBat_libprep,"[[2778.5973191368726, 9.714860985983748], [193...",NaN,NaN,NaN,NaN
AN_libprep,"[[1232.8078794275993, 5.633040009423116], [194...",NaN,NaN,NaN,NaN
EB_libprep,"[[2776.0425094020993, 9.71337176854651], [1933...",NaN,NaN,NaN,NaN
MP_libprep,"[[-2491.2859915329473, -9.032886088327409], [-...",NaN,NaN,NaN,NaN
ComBatseq_mission,"[[3017.996870286447, 12.727298011010507], [328...",NaN,NaN,NaN,NaN
ComBat_mission,"[[3229.964765517151, 12.779760463845085], [272...",NaN,NaN,NaN,NaN
AN_mission,"[[1483.6950036457579, 6.25798358736889], [1052...",NaN,NaN,NaN,NaN
EB_mission,"[[3310.820725125439, 13.18610293409333], [2695...",NaN,NaN,NaN,NaN
MP_mission,"[[-1466.0965631037077, -5.393304087408006], [-...",NaN,NaN,NaN,NaN


In [121]:
# PCA
i=0
for x in pca_vector:
    
    vectors['PCA'].loc[vectors.index[i]] = x
    
    i+=1

In [122]:
vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,"[[2128.978565204061, 10.478337653491538], [273...","[[-72.13313817411947, 77.03286680161123], [-83...",NaN,NaN,NaN
ComBat_libprep,"[[2778.5973191368726, 9.714860985983748], [193...","[[-72.03972280269909, 76.65703859598875], [-84...",NaN,NaN,NaN
AN_libprep,"[[1232.8078794275993, 5.633040009423116], [194...","[[-95.92866702291852, 109.53935538474386], [-1...",NaN,NaN,NaN
EB_libprep,"[[2776.0425094020993, 9.71337176854651], [1933...","[[-71.05433292096208, 75.4159071520525], [-83....",NaN,NaN,NaN
MP_libprep,"[[-2491.2859915329473, -9.032886088327409], [-...","[[-217.65887507126098, 243.30980589985984], [-...",NaN,NaN,NaN
ComBatseq_mission,"[[3017.996870286447, 12.727298011010507], [328...","[[-149.4935538388209, 78.21791669097645], [-80...",NaN,NaN,NaN
ComBat_mission,"[[3229.964765517151, 12.779760463845085], [272...","[[-95.63471711492227, 78.61905769685373], [-80...",NaN,NaN,NaN
AN_mission,"[[1483.6950036457579, 6.25798358736889], [1052...","[[-129.32529049924892, 107.07171698939271], [-...",NaN,NaN,NaN
EB_mission,"[[3310.820725125439, 13.18610293409333], [2695...","[[-67.89753595471728, 75.48748681304278], [-76...",NaN,NaN,NaN
MP_mission,"[[-1466.0965631037077, -5.393304087408006], [-...","[[-305.8069689805409, 254.3784674311712], [-25...",NaN,NaN,NaN


In [123]:
# DSC
i=0
for x in dsc_vector:
    
    vectors['DSC'].loc[vectors.index[i]] = x
    
    i+=1

In [124]:
vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,"[[2128.978565204061, 10.478337653491538], [273...","[[-72.13313817411947, 77.03286680161123], [-83...","[0.0291866500097831, 0.03396085434889798, 0.03...",NaN,NaN
ComBat_libprep,"[[2778.5973191368726, 9.714860985983748], [193...","[[-72.03972280269909, 76.65703859598875], [-84...","[0.002593068280833291, 0.019235656788409994, 0...",NaN,NaN
AN_libprep,"[[1232.8078794275993, 5.633040009423116], [194...","[[-95.92866702291852, 109.53935538474386], [-1...","[0.0365692443611619, -0.097572430740337, 0.119...",NaN,NaN
EB_libprep,"[[2776.0425094020993, 9.71337176854651], [1933...","[[-71.05433292096208, 75.4159071520525], [-83....","[0.0705058513837436, -0.219261778489742, 0.029...",NaN,NaN
MP_libprep,"[[-2491.2859915329473, -9.032886088327409], [-...","[[-217.65887507126098, 243.30980589985984], [-...","[0.007894540706487096, -0.226945515342322, 0.0...",NaN,NaN
ComBatseq_mission,"[[3017.996870286447, 12.727298011010507], [328...","[[-149.4935538388209, 78.21791669097645], [-80...","[-0.06132827018437571, -0.10016512503951303, -...",NaN,NaN
ComBat_mission,"[[3229.964765517151, 12.779760463845085], [272...","[[-95.63471711492227, 78.61905769685373], [-80...","[0.0383720473302919, 0.06183281432372699, 0.02...",NaN,NaN
AN_mission,"[[1483.6950036457579, 6.25798358736889], [1052...","[[-129.32529049924892, 107.07171698939271], [-...","[-0.2045898953888467, 0.11711067886384278, 0.0...",NaN,NaN
EB_mission,"[[3310.820725125439, 13.18610293409333], [2695...","[[-67.89753595471728, 75.48748681304278], [-76...","[-0.16949847402453372, 0.1382328450505605, 0.0...",NaN,NaN
MP_mission,"[[-1466.0965631037077, -5.393304087408006], [-...","[[-305.8069689805409, 254.3784674311712], [-25...","[0.014348952082065894, -0.217382267299087, 0.0...",NaN,NaN


In [125]:
# LFC
i=0
for x in lfc_vector:
    
    vectors['LFC'].loc[vectors.index[i]] = x
    
    i+=1

In [126]:
vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,"[[2128.978565204061, 10.478337653491538], [273...","[[-72.13313817411947, 77.03286680161123], [-83...","[0.0291866500097831, 0.03396085434889798, 0.03...","[0.007160493827160493, 0.0006172839506172847, ...",NaN
ComBat_libprep,"[[2778.5973191368726, 9.714860985983748], [193...","[[-72.03972280269909, 76.65703859598875], [-84...","[0.002593068280833291, 0.019235656788409994, 0...","[0.0069135802469135815, 0.01037037037037037, 0...",NaN
AN_libprep,"[[1232.8078794275993, 5.633040009423116], [194...","[[-95.92866702291852, 109.53935538474386], [-1...","[0.0365692443611619, -0.097572430740337, 0.119...","[0.007160493827160493, 0.008641975308641976, 0...",NaN
EB_libprep,"[[2776.0425094020993, 9.71337176854651], [1933...","[[-71.05433292096208, 75.4159071520525], [-83....","[0.0705058513837436, -0.219261778489742, 0.029...","[0.007160493827160495, 0.011111111111111113, 0...",NaN
MP_libprep,"[[-2491.2859915329473, -9.032886088327409], [-...","[[-217.65887507126098, 243.30980589985984], [-...","[0.007894540706487096, -0.226945515342322, 0.0...","[0.0064197530864197536, -0.0013580246913580227...",NaN
ComBatseq_mission,"[[3017.996870286447, 12.727298011010507], [328...","[[-149.4935538388209, 78.21791669097645], [-80...","[-0.06132827018437571, -0.10016512503951303, -...","[0.008395061728395062, -0.0012345679012345677,...",NaN
ComBat_mission,"[[3229.964765517151, 12.779760463845085], [272...","[[-95.63471711492227, 78.61905769685373], [-80...","[0.0383720473302919, 0.06183281432372699, 0.02...","[0.011604938271604939, 0.0029629629629629646, ...",NaN
AN_mission,"[[1483.6950036457579, 6.25798358736889], [1052...","[[-129.32529049924892, 107.07171698939271], [-...","[-0.2045898953888467, 0.11711067886384278, 0.0...","[0.003456790123456791, -0.006296296296296296, ...",NaN
EB_mission,"[[3310.820725125439, 13.18610293409333], [2695...","[[-67.89753595471728, 75.48748681304278], [-76...","[-0.16949847402453372, 0.1382328450505605, 0.0...","[0.01012345679012346, 0.002098765432098764, -0...",NaN
MP_mission,"[[-1466.0965631037077, -5.393304087408006], [-...","[[-305.8069689805409, 254.3784674311712], [-25...","[0.014348952082065894, -0.217382267299087, 0.0...","[0.005432098765432098, 0.004814814814814815, -...",NaN


In [129]:
# DGE across
i=0
for x in [list(x) for x in dge_across_vector]: # convert to list to be consistent
    
    vectors['DGEacross'].loc[vectors.index[i]] = x
    
    i+=1

In [130]:
vectors

,BatchQC,PCA,DSC,LFC,DGEacross
ComBatseq_libprep,"[[2128.978565204061, 10.478337653491538], [273...","[[-72.13313817411947, 77.03286680161123], [-83...","[0.0291866500097831, 0.03396085434889798, 0.03...","[0.007160493827160493, 0.0006172839506172847, ...","[6.067754262198707, 9.35270919067215, 6.403341..."
ComBat_libprep,"[[2778.5973191368726, 9.714860985983748], [193...","[[-72.03972280269909, 76.65703859598875], [-84...","[0.002593068280833291, 0.019235656788409994, 0...","[0.0069135802469135815, 0.01037037037037037, 0...","[4.837595532039978, 9.078619330676945, 6.33389..."
AN_libprep,"[[1232.8078794275993, 5.633040009423116], [194...","[[-95.92866702291852, 109.53935538474386], [-1...","[0.0365692443611619, -0.097572430740337, 0.119...","[0.007160493827160493, 0.008641975308641976, 0...","[7.649911816578484, 10.08969397086681, 5.30827..."
EB_libprep,"[[2776.0425094020993, 9.71337176854651], [1933...","[[-71.05433292096208, 75.4159071520525], [-83....","[0.0705058513837436, -0.219261778489742, 0.029...","[0.007160493827160495, 0.011111111111111113, 0...","[6.895208700764254, 12.173830208809633, 8.6670..."
MP_libprep,"[[-2491.2859915329473, -9.032886088327409], [-...","[[-217.65887507126098, 243.30980589985984], [-...","[0.007894540706487096, -0.226945515342322, 0.0...","[0.0064197530864197536, -0.0013580246913580227...","[9.373162845385066, 16.61536242297559, 11.6589..."
ComBatseq_mission,"[[3017.996870286447, 12.727298011010507], [328...","[[-149.4935538388209, 78.21791669097645], [-80...","[-0.06132827018437571, -0.10016512503951303, -...","[0.008395061728395062, -0.0012345679012345677,...","[1.4711934156378592, 2.389090295468896, 2.5269..."
ComBat_mission,"[[3229.964765517151, 12.779760463845085], [272...","[[-95.63471711492227, 78.61905769685373], [-80...","[0.0383720473302919, 0.06183281432372699, 0.02...","[0.011604938271604939, 0.0029629629629629646, ...","[4.211493239271016, 4.81508698586888, 3.001543..."
AN_mission,"[[1483.6950036457579, 6.25798358736889], [1052...","[[-129.32529049924892, 107.07171698939271], [-...","[-0.2045898953888467, 0.11711067886384278, 0.0...","[0.003456790123456791, -0.006296296296296296, ...","[1.6909171075837743, 2.0158213033727437, 0.681..."
EB_mission,"[[3310.820725125439, 13.18610293409333], [2695...","[[-67.89753595471728, 75.48748681304278], [-76...","[-0.16949847402453372, 0.1382328450505605, 0.0...","[0.01012345679012346, 0.002098765432098764, -0...","[4.16005291005291, 4.3925414244344285, 2.64880..."
MP_mission,"[[-1466.0965631037077, -5.393304087408006], [-...","[[-305.8069689805409, 254.3784674311712], [-25...","[0.014348952082065894, -0.217382267299087, 0.0...","[0.005432098765432098, 0.004814814814814815, -...","[17.263374485596707, 31.917036819300197, 17.97..."


In [131]:
vectors.to_csv('Scoring_Table.csv')